In [1]:
import pandas as pd
import numpy as np
import random

In [10]:
behaviors = pd.read_table('behaviors.tsv')
behaviors.set.value_counts()

set
train         156965
validation     73152
Name: count, dtype: int64

In [13]:
news = pd.read_table('news.tsv')
news.head()
news.set.value_counts()

set
train         51282
validation    42416
Name: count, dtype: int64

count of nulls

In [16]:
behaviors.isna().sum()

impression_id       0
user_id             0
time                0
history          5452
impressions         0
set                 0
dtype: int64

In [19]:
news.isna().sum()

news_id                 0
category                0
subcategory             0
title                   0
abstract             4687
url                     0
title_entities          5
abstract_entities       6
set                     0
dtype: int64

some news ids appear in the testing and validation sets

In [22]:
news.groupby('news_id').apply(lambda x: len(np.unique(x.set))).sort_values(ascending=False).head()
# news[news['news_id'] == 'N10']

news_id
N28801    2
N32734    2
N32750    2
N32749    2
N32748    2
dtype: int64

make each news article appear only in the train or validation set

In [25]:
def getSet(identifier, original, lkup):
    try:
        return lkup[identifier]
    except:
        return original
    

In [28]:
cnts = pd.DataFrame(news.groupby('news_id').apply(lambda x: len(np.unique(x.set))), columns=['cnt'])

# articles that appear in both train and validation
cntsGrtrThan1 = cnts[cnts['cnt'] > 1]

print('num articles that appear in both training and validation sets: ', cntsGrtrThan1.shape[0])

# generate random list whose values are 'train' or 'validation'
# the size of the list will be the same as the number of articles that appear in both train and validation
randChoices = random.choices(['train', 'validation'], k = cntsGrtrThan1.shape[0])

# dictionary from news_id to set (train or validation)
dic = dict(zip(cntsGrtrThan1.index, randChoices))

# fill the the new value for the 'set' column for each article
news['set'] = news.apply(lambda x: getSet(identifier = x['news_id'], original = x['set'],lkup=dic), axis=1)

# take distinct rows
news = news.drop_duplicates()

num articles that appear in both training and validation sets:  28460


In [79]:
news.groupby('news_id').apply(lambda x: len(np.unique(x.set))).sort_values(ascending=False).head()

news_id
N1        1
N49146    1
N49134    1
N49135    1
N49136    1
dtype: int64

In [80]:
news[news['news_id'] == 'N10']

,news_id,category,subcategory,title,abstract,url,title_entities,abstract_entities,set
155,N10,health,weight-loss,7 Running Workouts for Weight Loss,It's all about the fartleks.,https://assets.msn.com/labs/mind/AADnDn5.html,"[{""Label"": ""Exercise"", ""Type"": ""C"", ""WikidataI...",[],train


In [83]:
news.shape

(65238, 9)

some users appear in both training and validation sets

In [31]:
behaviors.groupby('user_id').apply(lambda x: len(np.unique(x.set))).sort_values(ascending=False).head()
# behaviors[behaviors['user_id'] == 'U9999']

user_id
U9999     2
U68503    2
U59344    2
U59343    2
U27973    2
dtype: int64

In [34]:
cnts = pd.DataFrame(behaviors.groupby('user_id').apply(lambda x: len(np.unique(x.set))), columns=['cnt'])

# users that appear in both train and validation
cntsGrtrThan1 = cnts[cnts['cnt'] > 1]

print('num users that appear in both training and validation sets: ', cntsGrtrThan1.shape[0])

# generate random list whose values are 'train' or 'validation'
# the size of the list will be the same as the number of users that appear in both train and validation
randChoices = random.choices(['train', 'validation'], k = cntsGrtrThan1.shape[0])

# dictionary from user_id to set (train or validation)
dic = dict(zip(cntsGrtrThan1.index, randChoices))

# fill the the new value for the 'set' column for each article
behaviors['set'] = behaviors.apply(lambda x: getSet(identifier = x['user_id']
                                                    , original = x['set']
                                                    ,lkup=dic)
                                                    , axis=1)


num users that appear in both training and validation sets:  5943


In [95]:
behaviors.head()
behaviors[behaviors['user_id'] == 'U9999']

,impression_id,user_id,time,history,impressions,set
122508,122509,U9999,11/10/2019 4:59:19 PM,N64503 N20039 N447 N10865 N59026 N12972 N25577...,N15224-0 N57651-0 N23184-0 N37533-0 N15931-0 N...,train
189965,33001,U9999,11/15/2019 5:38:10 PM,N64503 N20039 N447 N10865 N59026 N12972 N25577...,N52492-0 N54562-0 N4667-0 N64077-0 N56080-0 N3...,train
219147,62183,U9999,11/15/2019 6:20:18 PM,N64503 N20039 N447 N10865 N59026 N12972 N25577...,N30049-0 N512-0 N13887-0 N20764-0 N4667-0 N524...,train


In [98]:
print(behaviors.shape)
print(behaviors.drop_duplicates().shape)

(230117, 6)
(230117, 6)


get the number of clicks for each article

In [10]:
# clicks = behaviors.copy()

# clicks['history'] = clicks['history'].str.split(' ')

# clicks = clicks.explode('history')

# clicksAgg = clicks\
#                 .groupby('history')\
#                 .apply(lambda x: len(x))\
#                 .reset_index()\
#                 .rename(columns={0: 'num_clicks'
#                                 , 'history': 'news_id'})

# clicksAgg.head()


# # clicksAgg[clicksAgg['num_clicks'] > 0].head()

In [37]:
clicks = behaviors.copy()

clicks['impressions'] = clicks['impressions'].str.split(' ')

clicks = clicks.explode('impressions')

clicks['news_id'] = clicks['impressions'].apply(lambda x: x.split('-')[0])
clicks['num_of_clicks'] = clicks['impressions'].apply(lambda x: int(x.split('-')[1]))

clicksAgg = clicks\
                .groupby('news_id')\
                .agg({'num_of_clicks': 'sum'})\
                .reset_index()

clicksAgg.head()

,news_id,num_of_clicks
0,N10005,0
1,N10007,0
2,N10008,0
3,N10010,0
4,N10011,0


join number of clicks to news.tsv

In [40]:
news = news\
        .merge(clicksAgg
                , on='news_id'
                , how='left')


In [46]:
news.isna().sum()
# 42467

news_id                  0
category                 0
subcategory              0
title                    0
abstract              3415
url                      0
title_entities           3
abstract_entities        4
set                      0
num_of_clicks        42467
dtype: int64

In [49]:
behaviors.to_csv('behaviors_cleaned.tsv', index=False, sep='\t')
news.to_csv('news_cleaned.tsv', index=False, sep='\t')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c8be285d-9d64-40ba-873b-a353fe8ae087' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>